# SETUP

The first part of the code sets up the pca_wavelet network, the training comes later. Most of this code comes from the original authors

In [1]:
import sys
import time
sys.path.append('../segmentation_helper')

import tensorflow as tf
import keras
import tqdm
import matplotlib.pyplot as plt
import numpy as np
import data_loader as dl
import model_broker as mb
import os
import pandas as pd

tf.keras.backend.set_floatx("float64")

GPU device not found
Found GPU at: 


In [2]:
def scaledtanh(x): 
    return tf.math.tanh(x*0.5)

def scaledatanh(x):
    return tf.math.atanh(x)*2.0


In [ ]:
def conv_calculate_A_and_b(imghead, seghead, img_train,seg_train):
    n = 0.0

    imdecom_shape = imghead(next(iter(img_train))[0]).shape
    img_channels = imdecom_shape[3] # shape_0
    imdecom_2d_shape = imdecom_shape[1]*imdecom_shape[2] #shape_1

    seg_channels = segdecom_shape[3] # shape_0
    segdecom_2d_shape = segdecom_shape[1]*segdecom_shape[2] #shape_1
    segdecom_shape = seghead(next(iter(seg_train))[0]).shape

    xxt = np.zeros([img_channels,img_channels])
    yxt = np.zeros([img_channels,seg_channels])
    x = np.ones([imdecom_2d_shape])
    x_m = np.zeros([img_channels])
    y = np.ones([segdecom_2d_shape]) 
    y_m = np.zeros([seg_channels])

    bar = tqdm.notebook.tqdm(total = int(img_train.cardinality()))

    for item in iter(zip(img_train,seg_train)):
        bar.update(1)
        image = item[0][0]
        segmentation = item[1][0]

        imgdecom = imghead(image)
        segdecom = seghead(segmentation)

        mat = tf.reshape(imgdecom,[-1,seg_channels])
        segmat = tf.reshape(segdecom,[-1,img_channels])

        cov = tf.tensordot(mat,mat,[0,0])
        xxt += cov
        #del cov

        segcov = tf.tensordot(mat,segmat,[0,0])
        yxt += segcov
        #del segcov

        x_m += tf.linalg.matvec(mat,x,transpose_a=True)
        y_m += tf.linalg.matvec(segmat,y,transpose_a=True)

        n += 1
    return A,b

In [3]:
def connected_calculate_A_and_b(imghead, seghead, img_train,seg_train):
    imgflat = np.prod(imghead(next(iter(img_train))[0]).shape)
    segflat = np.prod(seghead(next(iter(seg_train))[0]).shape)
    end_shape = next(iter(seg_train))[0].shape
    n = 0.0

    xxt = np.zeros([imgflat])
    yxt = np.zeros([segflat])
    x = np.zeros([imgflat])
    y = np.zeros([segflat]) 

    bar = tqdm.notebook.tqdm(total = int(img_train.cardinality()))

    for item in iter(zip(img_train,seg_train)):

        bar.update(1)

        image = item[0][0]
        segmentation = item[1][0]

        imgdecom = imghead(image)
        segdecom = seghead(segmentation)

        mat = tf.reshape(imgdecom,[-1])
        segmat = tf.reshape(segdecom,[-1])

        cov = tf.matmul([mat],[mat],transpose_a=True)
        xxt += cov
        segcov = tf.matmul([mat],[segmat],transpose_a=True)
        yxt += segcov
        x+=mat
        y+=segmat
        n += 1
        
    print("loop calculated")
    xxt = xxt - tf.matmul([x],[x],transpose_a=True)/n
    yxt = yxt - tf.matmul([x],[y],transpose_a=True)/n
    print("calculating inverse")
    inverse_xxt = tf.linalg.pinv(xxt)
    print("calculating A")
    A = tf.linalg.matmul(inverse_xxt,yxt)
    print("calculating b")
    b = (y - tf.linalg.matvec(A,x,transpose_a=True))/n
    return A,b

In [4]:
def build_model_instance(train,
                test,
                dataset,
                model_name,
                keep_percent=1.0,
                count=3,
                sample_size=100,
                activity_regularizer=None,
                inverse_activity_regularizer=None,
                activation_before=False,
                check_build=False):
    
    stats = None
    
    broker = mb.ModelBroker(trainset=train,
                                testset=test,
                                dirname=dataset+"_"+model_name,
                                keep_percent=keep_percent,
                                count=count,
                                sample_size=sample_size,
                                activity_regularizer = activity_regularizer,
                                inverse_activity_regularizer=inverse_activity_regularizer,
                                activation_before=activation_before)
    
    head,invhead = broker.build_model()
    head,invhead = broker.load_model()    
    if check_build:
        train_psnr,train_ncc = broker.check_build(head,invhead,train,stats_only=True)
        test_psnr,test_ncc = broker.check_build(head,invhead,test,stats_only=True)
        stats = (train_psnr,train_ncc,test_psnr,test_ncc)
    return head,invhead, stats

In [13]:
def conv_metric_calculate():
    threshold_intensity = 0.01
    dice_coeff_vals = []
    iou_coeff_vals = []
    n = 0
    for image,seg_base in iter(zip(img_train,seg_train)):
        imgdecom = imghead(image[0])
        conv = tf.nn.conv2d(imgdecom, A_filter,1,"VALID")
        conv = tf.nn.bias_add(conv,b)
        seg = seginvhead(conv)
        y_true = tf.cast(tf.reduce_min(seg_base[0],2)==0,tf.float64)
        y_pred = tf.cast(tf.reduce_min(seg[0],2)<threshold_intensity,tf.float64)
        dice_coeff_vals.append(dice_coef(y_true,y_pred))
        iou_coeff_vals.append(iou_coef(y_true,y_pred))
        n+=1
    return iou_coeff_vals,dice_coeff_vals,n


In [14]:
def connected_metric_calculate(img_ds,seg_ds):
    threshold_intensity = 0.01
    dice_coeff_vals = []
    iou_coeff_vals = []
    n = 0
    reconstruct = seghead(next(iter(seg_ds))[0]).shape
    for image,seg_base in iter(zip(img_ds,seg_ds)):
        imgdecom = imghead(image[0])
        imgdecom = tf.reshape(imgdecom,(1,-1))
        segdecom = tf.linalg.matvec(A,imgdecom,transpose_a=True)+b
        seg = seginvhead(tf.reshape(segdecom,(reconstruct)))
        y_true = tf.cast(tf.reduce_min(seg_base[0],2)==0,tf.float64)
        y_pred = tf.cast(tf.reduce_min(seg[0],2)<threshold_intensity,tf.float64)
        dice_coeff_vals.append(dice_coef(y_true,y_pred))
        iou_coeff_vals.append(iou_coef(y_true,y_pred))
        n+=1
    return iou_coeff_vals,dice_coeff_vals,n

In [5]:
def calculate_metrics(seg_ds,img_ds,imghead,seghead,seginvhead):
    
    iou_coeff_vals,dice_coeff_vals,n = connected_metric_calculate(img_ds,seg_ds)
    #iou_coeff_vals,dice_coeff_vals,n = conv_metric_calculate(img_ds,seg_ds)
    dice_coeff_mean = sum(dice_coeff_vals)/n
    iou_coeff_mean = sum(iou_coeff_vals)/n
    dice_coeff_std = (sum([((x - dice_coeff_mean) ** 2) for x in dice_coeff_vals]) / n)**0.5
    iou_coeff_std = (sum([((x - iou_coeff_mean) ** 2) for x in iou_coeff_vals]) / n)**0.5
    return dice_coeff_mean, iou_coeff_mean, dice_coeff_std, iou_coeff_std

In [6]:
def dice_coef(y_true, y_pred,smooth=1):
    y_true_f = tf.reshape(y_true,-1)
    y_pred_f =tf.reshape(y_pred,-1)
    intersection = tf.reduce_sum(y_true_f * y_pred_f,0)

    return float((2. * intersection+smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)+smooth))

In [7]:
def iou_coef(y_true, y_pred,smooth=1):
  intersection = tf.reduce_sum(y_true * y_pred, 0)
  union = tf.reduce_sum(y_true,0)+tf.reduce_sum(y_pred,0)-intersection
  iou = tf.reduce_mean((intersection+1) / (union+1), 0)
  return float(iou)

In [9]:
counts_exp = [("count",i) for i in range(2,5)]
keep_percents_exp = [("keep_percent",i/10) for i in range(1,5)]
train_sizes_exp = [("train_size",i*2000) for i in range(1,4)]
res_exp = []#[("res",2**i) for i in range(6,9)]
experiments = counts_exp + keep_percents_exp + train_sizes_exp + res_exp

In [ ]:

test_size = 300

#These are the standard experiment settings
activity_regularizer = scaledtanh
inverse_activity_regularizer = scaledatanh
count = 3
keep_percent = 0.4
train_size = None
res = 128
dataset = "pets"

df = pd.DataFrame()

for settings in tqdm.notebook.tqdm(experiments):
    
    record = pd.Series()
    
    variable, value = settings
    if variable == "count":
        count = value
    if variable == "keep_percent":
        keep_percent = value
    if variable == "train_size":
        train_size = value
    if variable == "res":
        resolution = value

    loader = dl.DataLoader(IMAGE_SIZE=res,dataset=dataset,take=train_size)
    img_ds = loader.import_processed_img()
    seg_ds = loader.import_processed_seg()
    cardinality = int(img_ds.cardinality())

    img_test = img_ds.take(test_size)
    seg_test = seg_ds.take(test_size)
    img_train = img_ds.skip(test_size)
    seg_train = seg_ds.skip(test_size)    
    
    record["count"]=count
    record["keep_percent"] = keep_percent
    record["activity_regularizer"] = activity_regularizer != None
    record["training_data_size"] = train_size
    
    img_train_start = time.time()
    imghead, imginvhead,stats = build_model_instance(img_train,img_test,dataset,"img",keep_percent = keep_percent,count=count,check_build=True)
    psnr_train,ncc_train,psnr_test,ncc_test = stats
    img_train_end = time.time()
    
    record["img_channel_size"] = imghead(next(iter(img_train))[0]).shape[-1]
    record["img_train_time"] = img_train_end - img_train_start
    record["train_img_psnr"] = psnr_train
    record["train_img_ncc"] = ncc_train
    record["test_img_psnr"] = psnr_test
    record["test_img_ncc"] = ncc_test
    
    seg_train_start = time.time()
    seghead, seginvhead,stats = build_model_instance(seg_train,seg_test,dataset,"seg",count=count,keep_percent = keep_percent,check_build=True)
    psnr_train,ncc_train,psnr_test,ncc_test = stats
    seg_train_end = time.time()
    
    record["seg_channel_size"] = seghead(next(iter(seg_train))[0]).shape[-1]
    record["seg_train_time"] = img_train_end - img_train_start
    record["train_seg_psnr"] = psnr_train
    record["train_seg_ncc"] = ncc_train
    record["test_seg_psnr"] = psnr_test
    record["test_seg_ncc"] = ncc_test
    
    train_time_start = time.time()
    A,b = calculate_A_and_b(imghead,seghead,img_train,seg_train)
    train_time_end = time.time()
    
    record["linear_inverse_train_time"] = train_time_end - train_time_start 
    
    dice_mean_test, iou_mean_test, dice_std_test, iou_std_test = calculate_metrics(seg_test,img_test,imghead,seghead,seginvhead)
    dice_mean_train, iou_mean_train, dice_std_train, iou_std_train = calculate_metrics(seg_train,img_train,imghead,seghead,seginvhead)
    
    record["dice_mean_train"] = dice_mean_train
    record["iou_mean_train"] = iou_mean_train
    record["dice_std_train"] = dice_std_train
    record["iou_std_train"] = iou_std_train
    record["dice_mean_test"] = dice_mean_test
    record["iou_mean_test"] = iou_mean_test
    record["dice_std_test"] = dice_std_test
    record["iou_std_test"] = iou_std_test
    df = df.append(record,ignore_index=True)

  0%|          | 0/10 [00:00<?, ?it/s]

keep_percent 0.2810913475705226


<ipython-input-19-1d86426b1e64>:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  record = pd.Series()


meanimg.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
Starting level 0
Completing 64.0
pca shape tf.Tensor([27 27], shape=(2,), dtype=int32)
keep_channels 7 keep_max 12.0
keep_channels 7
ufilts.shape (1, 1, 1, 27, 7)
end loop 64.0
Starting level 1
Completing 32.0
pca shape tf.Tensor([63 63], shape=(2,), dtype=int32)
keep_channels 17 keep_max 112.0
keep_channels 17
ufilts.shape (1, 1, 1, 63, 17)
end loop 32.0
saving to: models/pets_img
out.shape (1, 32, 32, 17)
keep_percent 0.2810913475705226
meanimg.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
Starting level 0
Completing 64.0
pca shape tf.Tensor([27 27], shape=(2,), dtype=int32)
keep_channels 7 keep_max 12.0
keep_channels 7
ufilts.shape (1, 1, 1, 27, 7)
end loop 64.0
Starting level 1
Completing 32.0
pca shape tf.Tensor([63 63], shape=(2,), dtype=int32)
keep_channels 17 keep_max 112.0
keep_channels 17
ufilts.shape (1, 1, 1, 63, 17)
end loo

array([[[0.96515107, 0.93393552, 0.91092509],
        [0.87081516, 0.80595565, 0.79187489],
        [0.86642015, 0.82180369, 0.79532158],
        ...,
        [0.87063849, 0.82426375, 0.78263777],
        [0.87352365, 0.82447964, 0.79275811],
        [0.92949313, 0.89744228, 0.88557947]],

       [[0.79790419, 0.68389052, 0.64531207],
        [0.46615493, 0.18274452, 0.15653101],
        [0.44881091, 0.19862564, 0.15095837],
        ...,
        [0.62600482, 0.32333842, 0.20385215],
        [0.56797886, 0.26039273, 0.1665628 ],
        [0.82270169, 0.72857165, 0.68948424]],

       [[0.78860247, 0.64573139, 0.60463631],
        [0.29589367, 0.04779268, 0.03921569],
        [0.44276914, 0.14519331, 0.0877092 ],
        ...,
        [0.60813946, 0.29297164, 0.14303529],
        [0.70828694, 0.45093444, 0.32121632],
        [0.81302035, 0.73581445, 0.68740761]],

       ...,

       [[0.7795037 , 0.67046571, 0.654024  ],
        [0.26556277, 0.05079561, 0.04123727],
        [0.27025986, 0

array([[[0.04227678, 0.06537727, 0.03921569],
        [0.06662023, 0.11495649, 0.05612745],
        [0.03921569, 0.05370926, 0.03921569],
        ...,
        [0.03921569, 0.0662837 , 0.03921569],
        [0.03921569, 0.05812677, 0.03921569],
        [0.03921569, 0.0575394 , 0.03921569]],

       [[0.03921569, 0.05390529, 0.03921569],
        [0.15582132, 0.20696758, 0.10521695],
        [0.04115062, 0.06445336, 0.03921569],
        ...,
        [0.03921569, 0.06420276, 0.03921569],
        [0.03921569, 0.05081332, 0.03921569],
        [0.03921569, 0.04375838, 0.03921569]],

       [[0.04127724, 0.04414398, 0.03921569],
        [0.16459315, 0.24875201, 0.12933445],
        [0.05768756, 0.09986261, 0.04812539],
        ...,
        [0.03921569, 0.0641221 , 0.03921569],
        [0.03921569, 0.05971201, 0.03921569],
        [0.04267554, 0.08517373, 0.03921569]],

       ...,

       [[0.36728755, 0.56087774, 0.28557298],
        [0.29968479, 0.52660608, 0.18709932],
        [0.18146782, 0

array([[[0.04227678, 0.06537727, 0.03921569],
        [0.06662023, 0.11495649, 0.05612745],
        [0.03921569, 0.05370926, 0.03921569],
        ...,
        [0.03921569, 0.0662837 , 0.03921569],
        [0.03921569, 0.05812677, 0.03921569],
        [0.03921569, 0.0575394 , 0.03921569]],

       [[0.03921569, 0.05390529, 0.03921569],
        [0.15582132, 0.20696758, 0.10521695],
        [0.04115062, 0.06445336, 0.03921569],
        ...,
        [0.03921569, 0.06420276, 0.03921569],
        [0.03921569, 0.05081332, 0.03921569],
        [0.03921569, 0.04375838, 0.03921569]],

       [[0.04127724, 0.04414398, 0.03921569],
        [0.16459315, 0.24875201, 0.12933445],
        [0.05768756, 0.09986261, 0.04812539],
        ...,
        [0.03921569, 0.0641221 , 0.03921569],
        [0.03921569, 0.05971201, 0.03921569],
        [0.04267554, 0.08517373, 0.03921569]],

       ...,

       [[0.36728755, 0.56087774, 0.28557298],
        [0.29968479, 0.52660608, 0.18709932],
        [0.18146782, 0

keep_percent 0.2810913475705226
meanimg.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
self.mean.dtype <dtype: 'float64'>
Starting level 0


In [ ]:
df.to_csv("formal_experiment")

In [ ]:
import random
reconstruct = seghead(next(iter(seg_train))[0]).shape
threshold_intensity = 0.01
skip = random.randint(0,70)
image,seg_base = next(iter(zip(img_test.skip(skip),seg_test.skip(skip))))
imgdecom = imghead(image[0])
imgdecom = tf.reshape(imgdecom,(1,-1))
segdecom = tf.linalg.matvec(A,imgdecom,transpose_a=True)+b
seg = seginvhead(tf.reshape(segdecom,(reconstruct)))
y_true = tf.cast(tf.reduce_min(seg_base[0],2)==0,tf.float64)
y_pred = tf.cast(tf.reduce_min(seg[0],2)<threshold_intensity,tf.float64)
plt.subplot(2,1,1)
plt.imshow(np.hstack([image[0],seg_base[0],seg[0]]))
plt.subplot(2,1,2)
plt.imshow(np.hstack([y_true,y_pred]))
